In [ ]:
'''
Bulk convert XML to CSV Single Partition
Author:  Justin Brandenburg, Data Scientist
Email:   jbrandenburg@mapr.com

Requirements: change Pyspark python to Python 3.5

In Sandbox:
- upload zip file into /user/user01 using Hue
- zip file gets unzipped, rename file to rw_raw

In Sandbox as user01 at command line:
[user01@maprdemo ~]$ cd rw_raw
[user01@maprdemo rw_raw]$ ls -1 | wc -
[user01@maprdemo ]$ mkdir rw_XML_train
[user01@maprdemo rw_raw]$ cd ..
- copy raw data to new folder for xml conversion
[user01@maprdemo ~]$ cp -R rw_raw rw_XML_train/
- delete rw_raw in hue to clear up space 
- convert to xml
[user01@maprdemo ~]$ cd rw_XML_train
[user01@maprdemo rw_XML_train]$ find $1 -name "* *.dat" -type f -print0 | while read -d $'\0' f; do mv -v "$f" "${f// /_}"; done
[user01@maprdemo rw_XML_train]$ for f in *.dat; do mv -- "$f" "${f%.dat}.xml"; 
[user01@maprdemo ~]$ /opt/mapr/spark/spark-2.1.0/bin/spark-submit  --packages com.databricks:spark-xml_2.10:0.4.1 /user/user01/Sensor_ETLsparksubmit.py
'''

In [2]:
#PYSPARK Executable script
#import libraries
print ("Importing dependencies....")
import sys
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, DoubleType, FloatType, DateType, TimestampType
from pyspark.sql.functions import date_format, col, desc, udf, from_unixtime, unix_timestamp, date_sub, date_add, last_day
import time
print("Import complete.\n") 

Importing dependencies....
Import complete.

In [4]:
%%local
# Put some data on the cluster to work with
! hadoop fs -mkdir '/data/predictive-maintenance/rw_XML_train'

In [8]:
%%local
import subprocess

In [6]:
%%local
import os
for filename in os.listdir('tmp/data/rw_XML_train/'):
    if filename.endswith(".dat"): 
        #print filename
        localpath = 'tmp/data/rw_XML_train/' + filename
        targetpath = '/data/predictive-maintenance/rw_XML_train/' + filename
        subprocess.call(['hadoop', 'fs', '-put', localpath, targetpath])
        # print(os.path.join(directory, filename))
        #continue
    else:
        continue

#! hadoop fs -put 'tmp/data/part-00000-45866095-f76d-4f6c-ba2d-a07f0ab2dc04.csv' '/data/predictive-maintenance/training-set/part-00000-45866095-f76d-4f6c-ba2d-a07f0ab2dc04.csv'

2016_1_0_12_20160722 1402551049.dat
2016_1_0_12_20160722 1402561226.dat
2016_1_0_12_20160722 1402571378.dat
2016_1_0_12_20160722 1402581570.dat
2016_1_0_12_20160722 1402591977.dat
2016_1_0_12_20160722 1403002154.dat
2016_1_0_12_20160722 1403012586.dat
2016_1_0_12_20160722 1403022728.dat
2016_1_0_12_20160722 1403032875.dat
2016_1_0_12_20160722 1403043022.dat
2016_1_0_12_20160722 1403053164.dat
2016_1_0_12_20160722 1403063316.dat
2016_1_0_12_20160722 1403073453.dat
2016_1_0_12_20160722 1403083605.dat
2016_1_0_12_20160722 1403093762.dat
2016_1_0_12_20160722 1403103914.dat
2016_1_0_12_20160722 1403114071.dat
2016_1_0_12_20160722 1403124218.dat
2016_1_0_12_20160722 1403134360.dat
2016_1_0_12_20160722 1403144497.dat
2016_1_0_12_20160722 1403154649.dat
2016_1_0_12_20160722 1403164831.dat
2016_1_0_12_20160722 1403174983.dat
2016_1_0_12_20160722 1403185131.dat
2016_1_0_12_20160722 1403195268.dat
2016_1_0_12_20160722 1403205420.dat
2016_1_0_12_20160722 1403215562.dat
2016_1_0_12_20160722 1403225

2016_1_0_12_20160722 1434235550.dat
2016_1_0_12_20160722 1434245747.dat
2016_1_0_12_20160722 1434255898.dat
2016_1_0_12_20160722 1434266054.dat
2016_1_0_12_20160722 1434276241.dat
2016_1_0_12_20160722 1434286408.dat
2016_1_0_12_20160722 1434296575.dat
2016_1_0_12_20160722 1434306758.dat
2016_1_0_12_20160722 1434316935.dat
2016_1_0_12_20160722 1434327137.dat
2016_1_0_12_20160722 1434337339.dat
2016_1_0_12_20160722 1434347511.dat
2016_1_0_12_20160722 1434357683.dat
2016_1_0_12_20160722 1434367875.dat
2016_1_0_12_20160722 1434378067.dat
2016_1_0_12_20160722 1434388264.dat
2016_1_0_12_20160722 1434398466.dat
2016_1_0_12_20160722 1434408663.dat
2016_1_0_12_20160722 1434418855.dat
2016_1_0_12_20160722 1434429047.dat
2016_1_0_12_20160722 1434439234.dat
2016_1_0_12_20160722 1434449451.dat
2016_1_0_12_20160722 1434459608.dat
2016_1_0_12_20160722 1434469800.dat
2016_1_0_12_20160722 1434479977.dat
2016_1_0_12_20160722 1434490144.dat
2016_1_0_12_20160722 1434500336.dat
2016_1_0_12_20160722 1434510

In [1]:
print ("Importing dependencies....")
import sys
import os

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1500400165929_0613,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
Importing dependencies....

In [ ]:
def xmlConvert(spark):

    etl_time = time.time()
    df = spark.read.format('com.databricks.spark.xml').options(rowTag='HistoricalTextData').load('maprfs:///user/user01/rw_XML_train')
    df = df.withColumn("TimeStamp", df["TimeStamp"].cast("timestamp")).groupBy("TimeStamp").pivot("TagName").sum("TagValue").na.fill(0)
    df.repartition(1).write.csv("maprfs:///user/user01/rw_etl.csv", header=True, sep=",")
    print ("Time taken to do xml transformation: --- %s seconds ---" % (time.time() - etl_time))

In [3]:
! hadoop fs -ls

Found 1 items
drwxr-xr-x   - 5004 5005          1 2018-02-01 03:16 .sparkStaging


In [1]:
n

NameError: name 'n' is not defined